ModuleNotFoundError: No module named 'torch'

In [2]:
import cv2

img = cv2.imread(r'C:\Users\au783153\OBM\CODES\HeathlandSpeciesClassifier\dataset\images\lidl\amm\im50_69_.1.0.jpg', cv2.IMREAD_GRAYSCALE)
sift = cv2.SIFT_create()
keypoints, descriptors = sift.detectAndCompute(img, None)

img_with_keypoints = cv2.drawKeypoints(img, keypoints, None)
cv2.imshow("SIFT Keypoints", img_with_keypoints)
cv2.waitKey(0)
cv2.destroyAllWindows()


ModuleNotFoundError: No module named 'cv2'

In [3]:
import numpy as np

# Normalize the array between 0 and 1 using broadcasting
arr = np.array([2, 4, 6, 8, 10])
normalized_arr = (arr - arr.min()) / (arr.max() - arr.min())

print(normalized_arr)


ModuleNotFoundError: No module named 'numpy'